In [25]:
import json
import os
import math
import random
import numpy as np
from tqdm import tqdm


routing_name = "star16"
topology_name = "start_topo16_100000000000"

B = 12_500_000_000 # BYTES

# name of produces workload file
out_name = f'traces/trace_{routing_name}'       
# path to switch config files for topology                 
config_path = os.getcwd() + f'/topologies/{routing_name}/'       
# path to file containing topology
data_path = f'data/{topology_name}.json'

In [26]:
configs = os.listdir(config_path)

with open(data_path,'r') as f:
    data = json.load(f)

nodes = {}
for node in data['nodeList']:
    nodes[node['id']] = node

sources = [] # access
destinations = [] # kernel or mixed 

for config in configs:
    id = int(config.split('.')[0])
    
    if nodes[id]['deviceLevel'] == "Access":
        sources.append(id)
    else:
        destinations.append(id) 

print("num src:",len(sources), "|| num dests:",len(destinations))

num src: 16 || num dests: 1


In [41]:
FLOW_THROUGHPUT = B                     # BYTES PER SECOND
SIMULATION_TIME = 100_000_000               # Ns
PAIRS_PER_SRC = {'mu': 1, 'sigma': 0}   # NORMAL DIST
MSG_SIZE = 50_000                       # BYTES
PACKET_SIZE = 1400                      # BYTES
BANDWIDTH = 12_500_000_000                  # BYTES
PRIO_LEVELS = 3                         # QOS PRIORITIES

s_to_ns = lambda x : int(x * math.pow(10,9))
ns_to_s = lambda x : x * math.pow(10,-9)

In [35]:
parameters = (f"FLOW_THROUGHPUT_{FLOW_THROUGHPUT}__"
          f"SIMULATION_TIME_{SIMULATION_TIME}__"
          #f"PAIRS_PER_SRC_{tuple(PAIRS_PER_SRC.values())}__"
          f"MSG_SIZE_{MSG_SIZE}__"
          f"PACKET_SIZE_{PACKET_SIZE}__"
          f"BANDWIDTH_{BANDWIDTH}__"
          f"PRIO_LEVELS_{PRIO_LEVELS}")

print(parameters)

FLOW_THROUGHPUT_12500000000__SIMULATION_TIME_1000000000__MSG_SIZE_50000__PACKET_SIZE_1400__BANDWIDTH_12500000000__PRIO_LEVELS_3


In [36]:
# generate flows
def generate_flows():
    flows = {}
    for src in sources:
        num_pairs = int(random.normalvariate(**PAIRS_PER_SRC))
        while num_pairs <= 0:
            num_pairs = int(random.normalvariate(**PAIRS_PER_SRC))
        
        flows[src] = random.sample(destinations, k=num_pairs)
        
    return flows

flows = generate_flows()

In [37]:
def generate_messages_for_flow(duration_ns):
    total_bytes_for_duration = ns_to_s(duration_ns) * FLOW_THROUGHPUT

    sizes = []

    # generate message sizes using poisson distribution
    while sum(sizes) < total_bytes_for_duration:
        sizes.append(int(random.expovariate(1/MSG_SIZE)))
    
    # distribute the message send times uniformly over the simulation time
    times = np.linspace(0, duration_ns, len(sizes))
    times = [int(x) for x in times]

    return times, sizes

In [42]:
'''
    Generation Logic:
        - For each source - while the current message time is less than the simulation time
            - pick the next message time (exp dist)
            - pick a random dest (uniform)
            - pick a random message (size)
            - split the message into packets based on packets size
            - add packets accumulating transmission time (terminal -> switch)
'''

message_id = 0
messages = []
for src, pairs in tqdm(flows.items()):
     for dst in pairs:
        flow_messages = generate_messages_for_flow(SIMULATION_TIME)

        for time, size in zip(*flow_messages):
            tos = random.randint(0, PRIO_LEVELS-1)

            messages.append({
                'message_id': message_id,
                'src': src,
                'dst': dst,
                'size': size,
                'timestamp':time,
                'tos': tos
            })
        print("")
        message_id += 1

messages = sorted(messages, key=lambda x:x['timestamp'])

  6%|██▊                                         | 1/16 [00:00<00:14,  1.05it/s]

 12%|█████▌                                      | 2/16 [00:01<00:13,  1.02it/s]

 19%|████████▎                                   | 3/16 [00:02<00:12,  1.05it/s]

 25%|███████████                                 | 4/16 [00:03<00:11,  1.06it/s]

 31%|█████████████▊                              | 5/16 [00:04<00:10,  1.07it/s]

...

 75%|████████████████████████████████▎          | 12/16 [00:11<00:03,  1.08it/s]

 81%|██████████████████████████████████▉        | 13/16 [00:12<00:02,  1.07it/s]

 88%|█████████████████████████████████████▋     | 14/16 [00:13<00:01,  1.07it/s]

 94%|████████████████████████████████████████▎  | 15/16 [00:14<00:00,  1.08it/s]

100%|███████████████████████████████████████████| 16/16 [00:14<00:00,  1.07it/s]

In [44]:
len(messages)

399073

In [45]:
unique_packet_id = 0

i = 0
f =  open(out_name + "workload", 'w')

for msg in tqdm(messages):
    message_id, src, dst, size, timestamp, tos = msg.values()

    num_packets = math.ceil(size / PACKET_SIZE) # ceil -> padding last packet to always be PACKET_SIZE
    
    packet_time = timestamp
    packets = []
    for _ in range(num_packets):
        packet_info = (
            f"{str(unique_packet_id)} "
            f"{str(message_id)} "
            f"{str(src)} "
            f"{str(dst)} "
            f"{str(PACKET_SIZE)} "
            f"{str(packet_time)} "
            f"{str(tos)}\n"
        )

        f.write(packet_info)
        i = i+1
        packet_time += PACKET_SIZE / BANDWIDTH

        unique_packet_id += 1 

f.close()
print(i)
    

100%|████████████████████████████████| 399073/399073 [00:12<00:00, 32347.92it/s]

14486905
